In [4]:
import os
import sys
import json
from pprint import pprint

src_dir = os.path.join('..', 'src')
sys.path.append(os.path.abspath(src_dir))

from data import path, get_dataset

In [5]:
dataset = get_dataset()

../../data/datasets/dataset_1.00c.csv


In [6]:
files = list(os.listdir(path("comments")))

In [7]:
# score, all_awardings, body, downs, created, created_utc, depth, ups,

wanted_keys  =  set(['all_awardings', 'body', 'created', 'created_utc',
                     'depth', 'downs', 'id', 'parent_id', 'replies', 'score', 'ups', 'data', 'kind', 'children', 'name'])

list_keys = set(['all_awardings', 'children'])

def dict_struct(d, level=0):

    is_dict = isinstance(d, dict)
    is_list = isinstance(d, list)
    
    indent_str = ' .  '
    
    if is_dict:
    
        for k in sorted(d.keys()):
            if(k in wanted_keys):
                
                is_list_i = k in list_keys
                is_dict_i = isinstance(d[k], dict)
                if is_list_i or is_dict_i:
                    
                    print(indent_str * level, k, "= dict:" if is_dict_i else "")
                    dict_struct(d[k], level+1)
            
                else:
                    
                    if k == 'body':
                        print(indent_str * level, k, "=", d[k][:30], '...')
                    else:  
                        print(indent_str * level, k, "=", d[k])
        
    elif is_list:
        
        print(indent_str * level, '[')
        
        for i in d:
            dict_struct(i, level+1)
            if i != d[-1]:
                print(indent_str * level, '.   ' + ('= ' * int(0.5 * (100 - (level - 1) * len(indent_str)))))
        
        print(indent_str * level, ']')
    
    else:
        pass

In [8]:
comments_path = path("comments")

def post_dict_struct(post_id):
    
    file_path = os.path.join(comments_path, post_id + ".json")
    with open(file_path) as json_file:
        json_data = json.load(json_file)

        dict_struct(json_data)

In [9]:
post_dict_struct('laznue')

 [
 .   data = dict:
 .   .   children 
 .   .   .   [
 .   .   .   .   data = dict:
 .   .   .   .   .   all_awardings 
 .   .   .   .   .   .   [
 .   .   .   .   .   .   .   id = award_5f123e3d-4f48-42f4-9c11-e98b566d5897
 .   .   .   .   .   .   .   name = Wholesome
 .   .   .   .   .   .   ]
 .   .   .   .   .   created = 1612312024.0
 .   .   .   .   .   created_utc = 1612283224.0
 .   .   .   .   .   downs = 0
 .   .   .   .   .   id = laznue
 .   .   .   .   .   name = t3_laznue
 .   .   .   .   .   score = 174
 .   .   .   .   .   ups = 174
 .   .   .   .   kind = t3
 .   .   .   ]
 .   kind = Listing
 .   = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 
 .   data = dict:
 .   .   children 
 .   .   .   [
 .   .   .   .   data = dict:
 .   .   .   .   .   all_awardings 
 .   .   .   .   .   .   [
 .   .   .   .   .   .   ]
 .   .   .   .   .   body = First you give us Corner Gas,  ...
 .   .   .   .   .   created = 161231

In [10]:
import numpy as np

comments_path = path("comments")

def process_post(post_id):
    
    file_path = os.path.join(comments_path, post_id + ".json")
    with open(file_path) as json_file:
        json_data = json.load(json_file)

        return process_data_children_list(json_data)
    
    return []
  

# awards_keys = ['name']
def process_awards_list(awards):
    process = lambda x: "_".join(x.split(' '))
    return [process(elem["name"]) for elem in awards]


kinds = ['t1', 't3']
data_keys  =  ['all_awardings', 'body', 'created', 'created_utc', 'depth', 'downs', 'id', 'parent_id', 'score', 'ups']
def process_data_list(d):
    
    entities = []
    
    for elem in d:
    
        data = elem["data"]
        kind = elem["kind"]
        
        if kind not in kinds: 
            continue
            
        result = [kind]
    
        for k in data_keys:

            if k in data:

                if k == "all_awardings":
                    r = " ".join(process_awards_list(data[k]))
                elif k == "parent_id":
                    r = data[k].split('_')[1]
                else:
                    r = data[k]

            else:
                r = np.nan

            result.append(r)

        entities.append(tuple(result))

        if "replies" in data:
            replies = data["replies"]
            
            if replies != '':
                replies_result = process_data_children_list([replies])
                entities = entities + replies_result

    return entities
    
    
data_children_keys = ['data', 'kind']
def process_data_children_list(d):
    
    results = []
    for elem in d:

        for k in data_children_keys:

            if k == "data":
                children = elem[k]["children"]
                r = process_data_list(children)
                results = results + r
            
            if k == "kind":
                pass

    return results
        

In [15]:
import pandas as pd
import tqdm.notebook as tq

def process_posts_to_list(ids_to_process):

    results = []
    
    for i in tq.tqdm(range(len(ids_to_process))):
    
        id_ = ids_to_process[i]   
        
        res = process_post(id_)
        results.extend([tuple(list(x) + [id_]) for x in res])
    
    return results

In [22]:
def result_to_pandas(results):
    
    df = pd.DataFrame(results, columns=["kind"]+data_keys+["post_id"])

    order = ['id', 'parent_id', 'post_id', 'all_awardings', 'body', 'created', 'created_utc', 'kind', 'depth', 'score', 'ups', 'downs']

    types = {'id':'string', 'post_id':'string', 'parent_id':'string', 'all_awardings':'string', 'body':'string', 
             'created':np.float64,
             'created_utc':np.float64, 'kind':'string', 'depth':int, 'score':np.float64,
             'ups':int, 'downs':int}

    df = df[order]
    
    df.depth = np.where(np.isnan(df.depth), -1, df.depth)
#     df.parent_id = np.where(df.parent_id == "NaN", "N/A", df.parent_id)
#     df.body = np.where(df.body == "NaN", "N/A", df.body)
    df = df.astype(types)

    return df

In [18]:
result = process_posts_to_list(dataset.id.tolist())

In [23]:
df = result_to_pandas(result)

In [30]:
df[df.post_id == 'l0hnrw']

,id,parent_id,post_id,all_awardings,body,created,created_utc,kind,depth,score,ups,downs
0,l0hnrw,<NA>,l0hnrw,Helpful Wholesome Silver Hugz Snek,<NA>,1.611083e+09,1.611054e+09,t3,-1,600.0,600,0
1,gjtt5l2,l0hnrw,l0hnrw,All-Seeing_Upvote Mind_Blown Buy Party_Train S...,If you want to talk GME please use the specifi...,1.611092e+09,1.611063e+09,t1,0,1.0,1,0
2,gk7w5ga,l0hnrw,l0hnrw,,BlackBerry shares are trading higher on above...,1.611370e+09,1.611341e+09,t1,0,1.0,1,0
3,gjz4xdd,l0hnrw,l0hnrw,,Any one buying AFRM?,1.611199e+09,1.611170e+09,t1,0,2.0,2,0
4,gjy4ztb,l0hnrw,l0hnrw,,Good entry point for BNGO,1.611183e+09,1.611154e+09,t1,0,1.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
162,gjvhztp,gjvhyix,l0hnrw,,W.e it’s above 550 lol,1.611121e+09,1.611092e+09,t1,3,1.0,1,0
163,gjvhq69,l0hnrw,l0hnrw,,[usunięto],1.611120e+09,1.611092e+09,t1,0,3.0,3,0
164,gjvhued,gjvhq69,l0hnrw,,Rip,1.611120e+09,1.611092e+09,t1,1,3.0,3,0
165,gjvhluh,l0hnrw,l0hnrw,,Dare me to buy 2 call orders @14 1/22?,1.611120e+09,1.611092e+09,t1,0,5.0,5,0


In [31]:
df.to_csv(os.path.join(path("dataset"), "comments_v2.csv"), sep='`')

In [32]:
df[df.kind == 't3'].shape

(38908, 12)

In [33]:
list(set(df.kind.tolist()))

['t3', 't1']

In [35]:
from collections import Counter

awards = df.all_awardings.tolist()
res = []
for a in awards:
    res.extend(a.split(' '))
    
Counter(res).most_common()

[('', 795357),
 ('Silver', 7175),
 ('Helpful', 6291),
 ('Wholesome', 5864),
 ('Hugz', 5837),
 ('Gold', 3976),
 ('Take_My_Energy', 3218),
 ('All-Seeing_Upvote', 2958),
 ('To_The_Stars', 1793),
 ('Stonks_Rising', 1700),
 ('Party_Train', 1257),
 ("I'll_Drink_to_That", 1083),
 ('Ally', 992),
 ('Platinum', 918),
 ('Starstruck', 909),
 ('Starry', 890),
 ('LOVE!', 858),
 ('Narwhal_Salute', 834),
 ('Heartwarming', 705),
 ('This', 682),
 ('Wholesome_Seal_of_Approval', 672),
 ('Bravo!', 623),
 ('Faith_In_Humanity_Restored', 532),
 ('Burning_Cash', 514),
 ('Helpful_(Pro)', 481),
 ('Take_My_Power', 473),
 ('Bravo_Grande!', 416),
 ('Table_Slap', 366),
 ("'MURICA", 365),
 ('Eureka!', 354),
 ('Wearing_is_Caring', 309),
 ('Take_My_Money', 308),
 ('Snek', 307),
 ('hehehehe', 284),
 ('Rocket_Like', 264),
 ('Bless_Up', 244),
 ('Timeless_Beauty', 231),
 ('Buy', 219),
 ('Lawyer_Up', 217),
 ('Aww-some', 214),
 ('Buff_Doge', 207),
 ('Bless_Up_(Pro)', 202),
 ('Bull', 177),
 ('Crab_Rave', 163),
 ('Chad', 147),